# Crawling class tests

## Scholar crawling tests

In [1]:
import django
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "byteme.settings.base")
django.setup()

In [17]:
from crawler.models import Crawler
from accounts.models import Speaker, UserProfile
from events.tag import Tag
from events.models import Event

In [3]:
my_crawler = Crawler()

In [4]:
s1 = Speaker(name='Min H Kim', speakerEmail='1@kaist.ac.kr')
s2 = Speaker(name='Daniel Suk Jeon', speakerEmail='2@kaist.ac.kr')
s3 = Speaker(name='Osman', univ='Stanford', speakerEmail='3@kaist.ac.kr')
s1.save()
s2.save()
s3.save()
print(Speaker.objects.all())

<QuerySet [<Speaker: Min H Kim>, <Speaker: Daniel Suk Jeon>, <Speaker: Osman>]>


In [ ]:
s1 = Speaker.objects.all()[0]
s2 = Speaker.objects.all()[1]
s3 = Speaker.objects.all()[2]

In [7]:
my_crawler.scholar_crawl_request(s1)
my_crawler.scholar_crawl_request(s2)
my_crawler.scholar_crawl_request(s3)

scholar is not crawled
scholar is not crawled
scholar is not crawled
Entered Scholar Update
Will add keys
Succesfully added keys to scholar
Failed finding citation numbers
Entered Scholar Update
Will add keys
Succesfully added keys to scholar
Failed finding citation numbers
Failed finding paper author or confs
Entered Scholar Update
Will add keys
Succesfully added keys to scholar
Finished crawling :(


In [8]:
print(Tag.objects.all())
for speaker in Speaker.objects.all():
    print(speaker.tags.all())

<QuerySet [<Tag: Computer Graphics & Vision>, <Tag: Computational Imaging>, <Tag: Color>, <Tag: Computational Catalysis>, <Tag: Biomass catalysis>, <Tag: Machine Learning>, <Tag: CatApp>, <Tag: Chemical Database>]>
<QuerySet [<Tag: Computer Graphics & Vision>, <Tag: Computational Imaging>, <Tag: Color>]>
<QuerySet []>
<QuerySet [<Tag: Computational Catalysis>, <Tag: Biomass catalysis>, <Tag: Machine Learning>, <Tag: CatApp>, <Tag: Chemical Database>]>


## Indiv Score testing

In [19]:
print('UserProfiles: ', UserProfile.objects.all())
print('Speakers: ', Speaker.objects.all())
print('Tags:', Tag.objects.all())
print('Events:', Event.objects.all())

UserProfiles:  <QuerySet [<UserProfile: admin, KAIST>, <UserProfile: mustafa, KAIST>]>
Speakers:  <QuerySet [<Speaker: Min H. Kim>, <Speaker: Daniel Suk Jeon>, <Speaker: Osman Mamun>, <Speaker: Stephen Park>]>
Tags: <QuerySet [<Tag: Computer Graphics & Vision>, <Tag: Computational Imaging>, <Tag: Color>, <Tag: Computational Catalysis>, <Tag: Biomass catalysis>, <Tag: Machine Learning>, <Tag: CatApp>, <Tag: Chemical Database>, <Tag: Anaerobic Digestion>, <Tag: Microalgae>, <Tag: Computational Fluid Dynamics>]>
Events: <QuerySet [<Event: admin, KAIST Zombies>]>


In [35]:
event = Event.objects.all()[0]
user = UserProfile.objects.all()[0]

In [36]:
s1 = set(user.tags.all())
s2 = set(event.tags.all())
print(s1, s2, s1.intersection(s2))

{<Tag: Computational Imaging>} {<Tag: Computational Imaging>, <Tag: Computational Catalysis>, <Tag: Biomass catalysis>} {<Tag: Computational Imaging>}


In [42]:
def generateScore(user, event):
    intersection_max = 4
    user_tags = set(user.tags.all())
    event_tags = set(event.tags.all())
     
    intersection_num = min(intersection_max, len(s1.intersection(s2)))
    
    return intersection_num

In [43]:
generateScore(user, event)

1

## Pytrends Test

In [ ]:
import pytrends
from pytrends.request import TrendReq

pytrend = TrendReq(hl='en-US', tz=360)
#pytrend = TrendReq(hl='en-US', tz=360, proxies = {'https': 'https://34.203.233.13:80'})

In [ ]:
#kw_list = ["Blockchain"]
kw_list = ['generative adverserial network', 'neural machine translation', 'neural turing machine']
pytrend.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='')

In [ ]:
interest_over_time_df = pytrend.interest_over_time()
print(interest_over_time_df)

In [ ]:
def parse_scholar_id(in_str):
    idx1 = in_str.find('user=') + 5
    idx2 = in_str.find('&') #Check this!!
    substr = in_str[idx1:idx2]
    return substr

def create_link(scholar_id):
    return "http://scholar.google.com/citations?user=" + scholar_id + "&hl=en"

In [ ]:
query = "Dieter Fox google scholar"

matches = set()
for j in search(query, stop=5): 
    if "scholar.google" in j and len(j) < 100:
        scholar_id = parse_scholar_id(j)
        matches.add(scholar_id)
        
print('Number of links: ', len(matches))

In [ ]:
scholar_id = matches.pop()
link = create_link(scholar_id)
print(link)
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())